In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib


csv_path = "engine_data.csv"

# Load dataset from CSV
dn = pd.read_csv(csv_path)
# Encode categorical labels
label_encoder = LabelEncoder()
dn["Encoded_Status"] = label_encoder.fit_transform(
    dn[["Possible_Causes", "Recommended_Actions", "Engine_Status"]].apply(lambda x: " | ".join(x.astype(str)), axis=1)
)

# Splitting dataset
X = dn.drop(columns=["Possible_Causes", "Recommended_Actions", "Engine_Status", "Encoded_Status"])
y = dn["Encoded_Status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)





# Save the model
model_path = "engine_model_new.pkl"
joblib.dump(model, model_path)

# Load the model
loaded_model = joblib.load(model_path)

print(f"Model saved as {model_path}")

# Function to predict cause based on parameters
def predict_engine_issue(model, input_data):
    input_df = pd.DataFrame([input_data])
    prediction = model.predict(input_df)[0]
    decoded_label = label_encoder.inverse_transform([prediction])[0]
    return decoded_label


In [ ]:
 #Function to predict cause based on parameters

model_path = "engine_model_new.pkl"
def predict_engine_issue(model, input_data):
    input_df = pd.DataFrame([input_data])
    prediction = model.predict(input_df)[0]
    decoded_label = label_encoder.inverse_transform([prediction])[0]
    possible_cause, recommended_action, status = decoded_label.split(" | ")
    return {
        "Possible Cause": possible_cause,
        "Recommended Action": recommended_action,
        "Engine Status": status
    }

# Load the model
loaded_model = joblib.load(model_path)
# Load the model

def simulate():
    test_input = {
        "RPM": 1000,
        "Coolant_Temperature": 1110,
        "Oil_Pressure": 20,
        "Throttle_Position": 70,
        "Fuel_MPG": 5,
        "Compression_Ratio": 9,
        "Boost_Pressure": 10
    }

    result = predict_engine_issue(loaded_model, test_input)
    print("Simulation Result:", result)

simulate()


In [ ]:
from sklearn.metrics import accuracy_score

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2%}")
